In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_breast_cancer, load_digits

## breast cancer 분류, best model save, early stopping, graph, 2진분류

In [ ]:
bc = load_breast_cancer() 

In [ ]:
x = bc.data
y = bc.target

In [ ]:
model = Sequential([
    Dense(300, input_dim=30, activation='relu'),
    Dense(120, activation='relu'),
    Dense(60, activation='relu'),
    Dense(30, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(loss = 'binary_crossentropy',
             optimizer='nadam',
             metrics=['accuracy'])

In [ ]:
MODEL_DIR = '../model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [ ]:
modelpath = MODEL_DIR + 'cencer{epoch:03d}-{val_loss:.4f}.hdf5'
checkpointer_cb = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=1, save_best_only=True)
earlystopping_cb = EarlyStopping(monitor='val_loss', patience=100)

In [ ]:
history = model.fit(x, y, epochs=3000, validation_split=0.2, batch_size= 100, verbose=2, callbacks=[earlystopping_cb, checkpointer_cb])

In [ ]:
del model

In [ ]:
model = keras.models.load_model('../model/cencer908-0.0800.hdf5')

In [ ]:
model.evaluate(x, y)[1]

In [ ]:
y_vloss =history.history['val_loss']
y_acc = history.history['accuracy']

In [ ]:
x_len = np.arange(len(y_acc))
plt.figure(figsize=(12, 7))
plt.ylim(0.0, 1.1)
plt.plot(x_len, y_vloss, 'o', color='red', markersize=2)
plt.plot(x_len, y_acc, 'o', color='blue', markersize=2)
plt.show()

## load_digits = 다중분류 ->팀 회식
### random_seed = 2020, train/test 8:2 validation 0.2

In [ ]:
keras.backend.clear_session()

In [ ]:
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)

In [ ]:
digs = load_digits()

In [ ]:
x_data = digs.data.astype(np.float32)
y_data = digs.target

In [ ]:
labels = LabelEncoder().fit_transform(y_data)
y_data = np_utils.to_categorical(labels)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=seed)

In [ ]:
model = Sequential([
    Dense(250, input_dim=64, activation='selu', kernel_initializer='lecun_normal'),
    Dense(100, activation='selu', kernel_initializer='lecun_normal'),
    Dense(50, activation='selu', kernel_initializer='lecun_normal'),
    Dense(10, activation='softmax')
])

In [ ]:
earlystopping_cb = EarlyStopping(monitor='val_loss', patience=50)

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer = keras.optimizers.Adam(lr=0.006),
             metrics=['accuracy'])

In [ ]:
history = model.fit(x_train, y_train, epochs=3000, batch_size=25, validation_split=0.2, callbacks=[earlystopping_cb])

In [ ]:
model.evaluate(x_test, y_test)[1]

In [ ]:
y_vloss =history.history['val_loss']
y_acc = history.history['accuracy']

In [ ]:
x_len = np.arange(len(y_acc))
plt.figure(figsize=(12, 7))
plt.ylim(0.0, 1.1)
plt.plot(x_len, y_vloss, '-', color='red', markersize=2)
plt.plot(x_len, y_acc, '-', color='blue', markersize=2)
plt.show()